In [14]:
using Plots
using QuadGK
using NLopt
#using SpecialFunctions
using NLsolve
using Statistics
#include("../../scaling_CGS.jl")
include("../../code_to_real.jl")

find_real_units_HL (generic function with 2 methods)

In [15]:
#-------- From Dahldorf et al. 2014

c_code = 1.0
B_code = 0.05 #* sqrt(4*pi) # Multiply with sqrt(4*pi) to get it in CGS - they have in Heaviside-Lorentz units
p_code = 0.0002
rho_code = 1.0
debye_code = -1 #/ sqrt(4*pi) # divide with sqrt(4*pi) to get it in CGS - they have in Heaviside-Lorentz units
skin_depth_code = 1.0


c_u = code_units(c_code, B_code, p_code, rho_code, debye_code, skin_depth_code)

number_density_real = 1e9 # Guess 


r_u = known_real_units(number_density_real)
eps_0_scaling = 1e2
charge_scaling = 1e0
electron_proton_mass_ratio = 20

r_s = ratios_and_scaling(electron_proton_mass_ratio, eps_0_scaling, charge_scaling)


scaling = find_real_units_HL(c_u, r_u, r_s, true)

ds = 0.1
per_cell = 1024

set_macro_particle_weights(scaling, ds, per_cell)

print_basic_info(scaling)
print_all_HL(scaling)

lenght                 = 72.00847332647693
number_density         = 1.0e9
B                      = 6.281802035524741
T                      = 1.1432609021598859e7

length_scale           = 72.00847332647693
mass_density_scale     = 1.7562529929e-15
time_scale             = 2.401944125174654e-8
temperature_scale      = 1.0

e_mass_scale           = 91.80763368800336
charge_scaling         = 1.0
eps_0_scaling          = 100.0
mu_0_scaling           = 1.0

 Maxwell Lorents Factors used in code:
 k_E =                                             =  7.9577e-02 
 k_B =                                             =  7.9577e-02 
 k_F =                                             =  1.0000e+00 
 k_D =                                             =  1.0000e+00 
 k_M =                                             =  1.0000e+00 
 k_H =                                             =  1.0000e+00 

 Macro particle weights:
                          ds =                     =  1.0000e-01 
               

In [19]:
gamma = 5/3
c_f = sqrt((scaling.B_flux_code^2 + gamma *scaling.pressure_code) / scaling.mass_density_code)


Lx = 50
kx = 2*pi / Lx
Ly = 40
ky = 2*pi / Ly

k = sqrt(kx^2 + ky^2)

omega = c_f * k
wave_period = 1 / omega * 2pi

600 / wave_period

1.0224969437607139

In [ ]:
1 / (4.4721e+00 / 2pi)

In [ ]:
0.05 / scaling.weight_scaled_charge

In [ ]:
E0 = scaling.B_flux_code / 100

v_max = sqrt(2 * E0 / (scaling.m_e_code * scaling.number_density_code) )

In [ ]:
rho = scaling.mass_density_code
e_p_ratio = electron_proton_mass_ratio

rho_p = rho * e_p_ratio / (1 + e_p_ratio)
rho_e = rho / (1 + e_p_ratio)

In [ ]:
E0 = 0.001
n = scaling.number_density_code
q_e = -1 *  scaling.e_code
m = scaling.m_e_code


v_max = E0*  sqrt(  1.0 / rho_e)
dt_test = pi / (8* scaling.electron_plasma_freq_code)
v_test = E0 * q_e/m  * dt_test
v_test / v_max


t = 0:2.5e-2:4#range(0, stop=2.0, length=n_t)
n_t = length(t)
dt = t[2] - t[1]
v = zeros(n_t)
J = zeros(n_t)
E = zeros(n_t)
v[1] = v_test
E[1] = E0

for i in 2:n_t
    v[i] = v[i-1] + q_e/m * E[i-1] * dt
    J[i] = n * q_e * v[i]
    E[i] = E[i-1] - J[i] * dt
end


E_e = E.^2 / 2
E_p = 0.5 * n * m * v.^2;

In [ ]:
println("v_max = ", v_max)
println("v_test = ", v_test)
println("v_test / v_max = ", v_test / v_max)

In [ ]:
plot(t,v, label="v", size=(800,400))
plot!(t,E, label="E")
vline!([1.404974241895214/4, 1.404974241895214 , 2* 1.404974241895214])
hline!([-v_max, -E0, E0, v_max])

In [ ]:
plot(t,E_e, label="E_e")
plot!(t,E_p, label="E_p")

In [ ]:
E0 = 0.001
n = scaling.number_density_code
q_e = -1 *  scaling.e_code
m = scaling.m_e_code


v_max = E0*  sqrt(  1.0 / rho_e)
dt_test = pi / (8* scaling.electron_plasma_freq_code)
v_test = E0 * q_e/m  * dt_test
v_test / v_max


t = 0:2.5e-2:40#range(0, stop=2.0, length=n_t)
n_t = length(t)
dt = t[2] - t[1]
v = zeros(n_t)
J = zeros(n_t)
E = zeros(n_t)
v[1] = v_test*2
E[1] = E0

for i in 2:n_t
    v[i] = v[i-1] + q_e/m * E[i-1] * dt
    J[i] = n * q_e * v[i]
    E[i] = E[i-1] - J[i] * dt 
    E[i] = E[i] - E[i] * dt * 0.1
end


E_e = E.^2 / 2
E_p = 0.5 * n * m * v.^2;

plot(t,v, label="v", size=(800,400))
plot!(t,E, label="E")